In [1]:
from symbol import parameters

from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv
import warnings
import tiktoken
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from openai import OpenAI
import time
from src.embeddings import *
from src.data_processing import *
import tqdm
import itertools


warnings.filterwarnings("ignore")
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")


/var/folders/4p/xtbp10v53p19zbx0drx5xzcr0000gp/T/ipykernel_17364/3129549439.py:1: DeprecationWarning: The symbol module is deprecated and will be removed in future versions of Python
  from symbol import parameters


## Prepare Data

### Load Document

In [2]:
data = load_files("../Data/Raw")
print(len(data))


Processing files: 100%|██████████| 10/10 [00:14<00:00,  1.43s/it]

13660


In [3]:
data[100].keys()

dict_keys(['id', 'title', 'content', 'prechunk', 'postchunk', 'token_count'])

### Pinecone

In [6]:
vectors = get_dataset_embeddings_parallel(data)
len(vectors)

100%|██████████| 13660/13660 [10:31<00:00, 21.64it/s] 


13660

In [20]:
pc = Pinecone(api_key=pinecone_api_key)
index_name = "legal-assistant-rag"
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=3072,  # dimensionality of embed 3
        metric='cosine',
        spec=ServerlessSpec(
    cloud="aws", region="us-west-2"  # us-east-1
)
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
print(index.describe_index_stats())


def chunks(iterable, batch_size=200):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

for ids_vectors_chunk in chunks(vectors, batch_size=50):
    index.upsert(vectors=ids_vectors_chunk)

{'dimension': 3072,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}


In [21]:
print(index.describe_index_stats())

{'dimension': 3072,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 13660}},
 'total_vector_count': 13660}


In [19]:
len(vectors)

13660

In [1]:
query = "What are the main business requirements for Regulation K?"
xq = client.embeddings.create(input=query, model="text-embedding-3-large").data[0].embedding
res = index.query(vector=xq, top_k=10, include_metadata=True)
res

NameError: name 'client' is not defined